<img src="../images/QISKit-c.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="250 px" align="left">

## _*Bloch circuit annotation*_ <br />
© IBM Research

boilerplate
***
### Contributors
Andrew Cross

## Introduction

State tomography is a method for estimating the density matrix of a quantum state. A single-qubit state $\rho=(I+a_xX+a_yY+a_zZ)/2$ is fully determined by the expectation value of each Pauli operator. For example, $\langle X\rangle = \mathrm{Tr}(X\rho) = a_x$. The resulting vector $(a_x,a_y,a_z)$ is called the Bloch vector. Since $\rho$ is positive and $\mathrm{Tr}(\rho)=1$, $\mathrm{Tr}(\rho^2)\leq 1$. This implies that the Bloch vector lies on or within the unit sphere.

This notebook shows how to annotate a circuit with "bloch" gates to carry out single-qubit state tomography at select circuit locations. If the input circuit contains $m$ "bloch" gates, the make_bloch_circuits helper function makes a dictionary of $3m$ QASM strings to estimate the expectation values of $X$, $Y$, and $Z$ at each of the $m$ circuit locations. If you like, "bloch" gates can be viewed as a debugging tool for your quantum circuits.

For each occurrence of "bloch", the helper function (a) replaces the "bloch" gate with single-qubit gates and measurements, (b) deletes the future light cone of that "bloch" gate, and (c) removes all other "bloch" gates. All of the classical registers are renamed with "aaa" prepended. State tomography measurements are written to c[0]. If you run on the real hardware, there is only one creg, so you will need to delete "creg c[5];" from your input QASM as well as any measurements.

The examples below creates a Bell state $|\psi\rangle:=\frac{1}{\sqrt{2}}\left(|00\rangle+|11\rangle\right)$. We annotate the circuit to show that qubit q[0] begins in the state $|0\rangle$ on the north pole of the Bloch sphere and rotates to $|+\rangle$ on the equator. Qubits q[0] and q[1] interact via a CNOT and become the maximally entangled state $|\psi\rangle$. When you observe either q[0] or q[1] in isolation, they each appear to be in the maximally mixed state $\rho=I/2$ at the center of the Bloch sphere.

In [2]:
import sys
sys.path.append("..")
from qhelpers.basicplotter import plot_bloch_vector
from qhelpers import bloch
from IBMQuantumExperience import IBMQuantumExperience
import Qconfig

api = IBMQuantumExperience.IBMQuantumExperience(Qconfig.APItoken, Qconfig.config)

In [4]:
device = "simulator"  # simulator or real
shots = 1024
# Simulator inputs can contain cregs, measurements, and ifs
mysrcsim="""
OPENQASM 2.0;
include "qelib1.inc";
opaque bloch(p) q;
qreg q[5];
creg c[5];
bloch(1) q[0];
h q[0];
bloch(2) q[0];
cx q[0],q[1];
bloch(3) q[0];
bloch(4) q[1];
h q[2];
measure q[2] -> c[0];
if(c==1) x q[2];
"""

# Real device inputs cannot contain any cregs - your job will fail
mysrcreal="""
OPENQASM 2.0;
include "qelib1.inc";
opaque bloch(p) q;
qreg q[5];
bloch(1) q[0];
h q[0];
bloch(2) q[0];
cx q[0],q[1];
bloch(3) q[0];
bloch(4) q[1];
"""

# Generate and execute each experiment on the Quantum Experience
exps = bloch.make_bloch_circuits(mysrcsim)
print(exps)
#blochvecs = []
#for i in range(len(exps)):
#    results = []
#    for m in ["x","y","z"]:
#        expname=basename+"_%d_%s"%(i,m)
#        print("device=%s, shots=%s, expname=%s"%(device,shots,expname))
#        #print("%s\n"%exps[i][m])
#        out = api.runExperiment(exps[i][m],device,shots,expname)
#        #print("%s"%out)
#        results.append(out["result"]["measure"])
#    xdat = dict(zip(results[0]["labels"],results[0]["values"]))
#    ydat = dict(zip(results[1]["labels"],results[1]["values"]))
#    zdat = dict(zip(results[2]["labels"],results[2]["values"]))
#    #print("xdat=%s, ez=%f"%(xdat,bloch.ez(xdat,0)))
#    #print("ydat=%s, ez=%f"%(ydat,bloch.ez(ydat,0)))
#    #print("zdat=%s, ez=%f"%(zdat,bloch.ez(zdat,0)))
#    blochvecs.append((bloch.ez(xdat,0),bloch.ez(ydat,0),bloch.ez(zdat,0)))

## Report and plot the results
#print("bloch vectors =\n%s"%blochvecs)
#for i in range(len(blochvecs)):
#    print("bloch #%s, params=%s"%(i,",".join(exps[i]["key"])))
#    plot_bloch_vector(blochvecs[i])


Generating LALR tables
Generating LALR tables
Generating LALR tables
Generating LALR tables


bloch #0 -- qubit q[0], parameters ['1.0']
bloch #1 -- qubit q[0], parameters ['2.0']
bloch #2 -- qubit q[0], parameters ['3.0']
bloch #3 -- qubit q[1], parameters ['4.0']
[{'key': ['1.0'], 'x': 'IBMQASM 2.0;\ninclude "qelib1.inc";\nqreg q[5];\ncreg aaac[5];\ncreg c[5];\nu2(0.0,3.141592653589793) q[2];\nmeasure q[2] -> aaac[0];\nif(aaac==1) u3(3.141592653589793,0.0,3.141592653589793) q[2];\nu2(0.0,3.141592653589793) q[0];\nmeasure q[0] -> c[0];\n', 'y': 'IBMQASM 2.0;\ninclude "qelib1.inc";\nqreg q[5];\ncreg aaac[5];\ncreg c[5];\nu2(0.0,3.141592653589793) q[2];\nmeasure q[2] -> aaac[0];\nif(aaac==1) u3(3.141592653589793,0.0,3.141592653589793) q[2];\nu1(-1.5707963267948966) q[0];\nu2(0.0,3.141592653589793) q[0];\nmeasure q[0] -> c[0];\n', 'z': 'IBMQASM 2.0;\ninclude "qelib1.inc";\nqreg q[5];\ncreg aaac[5];\ncreg c[5];\nu2(0.0,3.141592653589793) q[2];\nmeasure q[2] -> aaac[0];\nif(aaac==1) u3(3.141592653589793,0.0,3.141592653589793) q[2];\nmeasure q[0] -> c[0];\n'}, {'key': ['2.0'], 'x': 